# Sandbox 2

In [ ]:
import pandas as pd
import configparser
import os
import jellyfish as jf
import py_stringmatching as sm
import numpy
from re import search

config_file = '../config.ini'
config = configparser.ConfigParser()
config.read(config_file)

In [ ]:
def is_combination(x):
    if search("combination", x):
        return 'Y'
    elif search(" and ", x):
        return 'Y'
    elif search("&", x):
        return 'Y'
    elif search(" with ", x):
        return 'Y'
    elif search("/", x):
        return 'Y'
    else:
        return 'N'

In [ ]:
mapped_atc_df = pd.read_csv(os.path.join('../output/atc_used.csv'),header=None)
mapped_atc_array = mapped_atc_df[0].to_numpy()
atc_df = pd.read_csv(os.path.join('../data/rxnorm_atc_code_info.csv'))
atc_df.columns = ['i', 'rxcui', 'rxaui', 'sab', 'tty', 'ATC code', 'ATC level name', 'suppress']
atc_df.drop('i', axis=1, inplace=True)
bnf_df = pd.read_csv(os.path.join('../output/bnf_code_clean.csv'))

In [ ]:
bnf_df

In [ ]:
atc_df['mapped_to_bnf'] = atc_df['ATC code'].apply(lambda x: ('N','Y')[x in mapped_atc_array])
atc_df['combination'] = atc_df.loc[atc_df['tty'].isin(['IN','RXN_IN'])]['ATC level name'].apply(lambda x: is_combination(x.lower()))
bnf_df['combination'] = bnf_df['bnf_chemical_substance'].apply(lambda x: is_combination(x.lower()))

In [ ]:
atc_df.loc[atc_df['ATC code'] == 'V04CC02']

In [ ]:
atc_df.loc[atc_df['mapped_to_bnf'] == 'Y']

In [ ]:
atc_df.loc[atc_df['combination'] == 'Y']

In [ ]:
atc_df.loc[atc_df['tty'].isin(['IN','RXN_IN'])].groupby('ATC level name').count()

In [ ]:
count_df = atc_df.groupby('ATC level name').count()
count_df.to_csv(os.path.join('../output/atc_count.csv'))
count_df = bnf_df.groupby('bnf_chemical_substance').count()
count_df.to_csv(os.path.join('../output/bnf_count.csv'))

In [ ]:
atc_df.to_csv(os.path.join('../output/atc_mapped_to_bnf.csv'))

In [ ]:
bnf_df.to_csv(os.path.join('../output/bnf_combination.csv'))